## This Python code simulates a warehouse with robots moving toward targets on a 10×10 grid. Robots may block each other, creating deadlocks. The system detects when no robot can move and uses BFS-based rerouting to free them. It prints grid states step by step for visualization.

By: **Akhileh Pan

In [2]:
import numpy as np
import random
from collections import deque

# Warehouse Configuration
GRID_SIZE = (10, 10)
NUM_ROBOTS = 5
STEPS = 50

# Directions: Up, Down, Left, Right
DIRECTIONS = [(0, 1), (0, -1), (-1, 0), (1, 0)]

# Initialize robots randomly
robot_positions = np.array([[random.randint(0, GRID_SIZE[0]-1),
                             random.randint(0, GRID_SIZE[1]-1)]
                            for _ in range(NUM_ROBOTS)])

# Target positions for robots
robot_targets = np.array([[random.randint(0, GRID_SIZE[0]-1),
                           random.randint(0, GRID_SIZE[1]-1)]
                          for _ in range(NUM_ROBOTS)])


def is_valid(pos, grid):
    """Check if a position is within grid and not occupied"""
    x, y = pos
    return 0 <= x < GRID_SIZE[0] and 0 <= y < GRID_SIZE[1] and grid[x, y] == -1


def move_robot(robot_id, grid):
    """Try to move robot towards its target"""
    x, y = robot_positions[robot_id]
    tx, ty = robot_targets[robot_id]

    # Calculate possible moves
    moves = []
    if tx > x:
        moves.append((x+1, y))
    elif tx < x:
        moves.append((x-1, y))
    if ty > y:
        moves.append((x, y+1))
    elif ty < y:
        moves.append((x, y-1))

    # Randomize moves to avoid conflicts
    random.shuffle(moves)

    for new_pos in moves:
        if is_valid(new_pos, grid):
            grid[x, y] = -1  # Free old position
            robot_positions[robot_id] = new_pos
            grid[new_pos[0], new_pos[1]] = robot_id
            return True
    return False


def detect_deadlock(grid):
    """Detect deadlock: no robot can move"""
    for robot_id in range(NUM_ROBOTS):
        x, y = robot_positions[robot_id]
        for dx, dy in DIRECTIONS:
            nx, ny = x + dx, y + dy
            if is_valid((nx, ny), grid):
                return False
    return True


def reroute_robot(robot_id, grid):
    """Escape deadlock by finding first available empty cell using BFS"""
    start = tuple(robot_positions[robot_id])
    queue = deque([start])
    visited = set([start])

    while queue:
        x, y = queue.popleft()
        if grid[x, y] == -1:
            # Move robot here
            ox, oy = robot_positions[robot_id]
            grid[ox, oy] = -1
            robot_positions[robot_id] = [x, y]
            grid[x, y] = robot_id
            return True
        for dx, dy in DIRECTIONS:
            nx, ny = x + dx, y + dy
            if 0 <= nx < GRID_SIZE[0] and 0 <= ny < GRID_SIZE[1] and (nx, ny) not in visited:
                visited.add((nx, ny))
                queue.append((nx, ny))
    return False


# Initialize grid: -1 means empty
grid = -1 * np.ones(GRID_SIZE, dtype=int)
for i, pos in enumerate(robot_positions):
    grid[pos[0], pos[1]] = i

# Simulation
for step in range(STEPS):
    print(f"Step {step+1}")
    deadlock_detected = False

    # Try moving each robot
    moved = [move_robot(rid, grid) for rid in range(NUM_ROBOTS)]

    if not any(moved):
        deadlock_detected = True

    # If deadlock, reroute robots
    if deadlock_detected or detect_deadlock(grid):
        print("Deadlock detected! Rerouting robots...")
        for rid in range(NUM_ROBOTS):
            reroute_robot(rid, grid)

    # Print grid
    for row in grid:
        print(" ".join(["." if x==-1 else str(x) for x in row]))
    print("\n")


Step 1
. . . . . . . . . .
. 4 . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . 2 . .
3 . . . . . . . . .
. . . . . . . . . .
. 1 . . . . . . . .
. . . . . 0 . . . .


Step 2
. . . . . . . . . .
. . 4 . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
3 . . . . . 2 . . .
. . . . . . . . . .
. 1 . . . . . . . .
. . . . . 0 . . . .
. . . . . . . . . .


Step 3
. . . . . . . . . .
. . . . . . . . . .
. . 4 . . . . . . .
. . . . . . . . . .
3 . . . . . . . . .
. . . . . 2 . . . .
. 1 . . . . . . . .
. . . . . 0 . . . .
. . . . . . . . . .
. . . . . . . . . .


Step 4
. . . . . . . . . .
. . . . . . . . . .
. . . 4 . . . . . .
3 . . . . . . . . .
. . . . . . . . . .
. 1 . . 2 . . . . .
. . . . . 0 . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .


Step 5
. . . . . . . . . .
. . . . . . . . . .
3 . . . . . . . . .
. . . 4 . . . . . .
. 1 . . . . . . . .
. . . 2 . 0 . . . .
. . . . . . . . . .
. . . . . . . . .

## **Step-by-Step Explanation of the Deadlock Detection & Avoidance System**
---

### **1. Setup the warehouse**

* A **10×10 grid** represents your warehouse.
* `-1` in a cell means **empty**, a number (0–4) means a **robot** is there.
* Each robot has a **random starting position** and a **random target position**.

```python
GRID_SIZE = (10, 10)
NUM_ROBOTS = 5
robot_positions = ...
robot_targets = ...
grid = -1 * np.ones(GRID_SIZE, dtype=int)
```

---

### **2. Robot movement**

* Each robot tries to move **closer to its target**.
* Robots can move **up, down, left, or right**.
* If a move is **blocked** (another robot or wall), it tries a different move.

```python
def move_robot(robot_id, grid):
    # Calculate possible moves towards the target
    # Move if the cell is free
```

* If no robot can move, the system may detect a **deadlock**.

---

### **3. Deadlock detection**

* A **deadlock** occurs when **no robot can move** in any direction.
* The function `detect_deadlock()` checks for this by trying all possible moves for all robots.

```python
def detect_deadlock(grid):
    # If every robot has no valid move, return True
```

---

### **4. Deadlock avoidance (rerouting)**

* When a deadlock is detected, the system tries to **reroute robots**.
* It uses a **BFS (Breadth-First Search)** to find the nearest empty cell for each robot.
* The robot moves there to **escape the deadlock**, freeing up space.

```python
def reroute_robot(robot_id, grid):
    # Find first empty cell using BFS and move the robot
```

---

### **5. Simulation loop**

* The code runs for a fixed number of steps (50).
* At each step:

  1. Robots try to move toward their targets.
  2. Deadlocks are checked.
  3. If deadlocked, rerouting is performed.
  4. The grid is printed showing **robot positions**.

```python
for step in range(STEPS):
    moved = [move_robot(rid, grid) for rid in range(NUM_ROBOTS)]
    if not any(moved) or detect_deadlock(grid):
        reroute_robot(...)
    print(grid)
```

---

### **6. Output**

* Each step prints the warehouse:

  * `.` → empty cell
  * `0, 1, 2, ...` → robot IDs
* When a deadlock occurs, it prints **“Deadlock detected! Rerouting robots…”** and moves robots to resolve the deadlock.

---

### **In short**

* **Move robots toward targets.**
* **Detect if they block each other.**
* **Reroute robots to avoid getting stuck.**
* **Visualize each step on the grid.**

---





### **Complete Code with Line-by-Line Explanation**

```python
import numpy as np  # For grid and array handling
import random       # For random positions of robots
from collections import deque  # For BFS in rerouting
```

* `numpy` → for creating and manipulating the warehouse grid.
* `random` → to randomly place robots and targets.
* `deque` → double-ended queue for efficient BFS search.

---

```python
GRID_SIZE = (10, 10)  # Warehouse size: 10x10
NUM_ROBOTS = 5         # Number of robots
STEPS = 50             # Number of simulation steps
```

* Defines **grid dimensions**, number of robots, and simulation length.

---

```python
# Initialize the warehouse grid
grid = -1 * np.ones(GRID_SIZE, dtype=int)  # -1 means empty cell
```

* Creates a **10×10 grid filled with -1** (all cells empty).
* Each robot will be assigned a number 0–4.

---

```python
# Initialize robots with random positions and random targets
robot_positions = {}
robot_targets = {}
for rid in range(NUM_ROBOTS):
    while True:
        pos = (random.randint(0, GRID_SIZE[0]-1), random.randint(0, GRID_SIZE[1]-1))
        if grid[pos] == -1:
            grid[pos] = rid
            robot_positions[rid] = pos
            break
    while True:
        target = (random.randint(0, GRID_SIZE[0]-1), random.randint(0, GRID_SIZE[1]-1))
        if target != pos:
            robot_targets[rid] = target
            break
```

* Assigns **each robot a unique random starting position** on the grid.
* Assigns a **random target position** for each robot (not same as start).
* Updates `grid` so we can visualize robot positions.

---

```python
def move_robot(rid, grid):
    """Move a robot one step toward its target if possible"""
    x, y = robot_positions[rid]
    tx, ty = robot_targets[rid]
    
    # Determine move directions prioritized by distance to target
    moves = []
    if tx > x: moves.append((x+1, y))
    if tx < x: moves.append((x-1, y))
    if ty > y: moves.append((x, y+1))
    if ty < y: moves.append((x, y-1))
    
    # Try moves in order
    for nx, ny in moves:
        if 0 <= nx < GRID_SIZE[0] and 0 <= ny < GRID_SIZE[1] and grid[nx, ny] == -1:
            grid[x, y] = -1         # Clear old position
            grid[nx, ny] = rid      # Move robot
            robot_positions[rid] = (nx, ny)
            return True  # Successfully moved
    return False  # Could not move
```

* Moves a robot **toward its target** step by step.
* Chooses direction **closest to the target**.
* **Checks bounds** and if the **cell is free**.
* Updates the grid and robot position if moved.

---

```python
def detect_deadlock(grid):
    """Check if all robots are blocked"""
    for rid in range(NUM_ROBOTS):
        if move_robot(rid, grid):  # If at least one robot can move
            return False
    return True  # None can move, deadlock detected
```

* Tests if **all robots are stuck**.
* Returns **True** if deadlock, **False** otherwise.

---

```python
def reroute_robot(rid, grid):
    """Reroute robot using BFS to nearest empty cell"""
    visited = set()
    queue = deque([robot_positions[rid]])
    
    while queue:
        x, y = queue.popleft()
        if grid[x, y] == -1:  # Found empty cell
            ox, oy = robot_positions[rid]
            grid[ox, oy] = -1
            grid[x, y] = rid
            robot_positions[rid] = (x, y)
            return
        # Explore neighbors
        for nx, ny in [(x+1,y),(x-1,y),(x,y+1),(x,y-1)]:
            if 0 <= nx < GRID_SIZE[0] and 0 <= ny < GRID_SIZE[1] and (nx, ny) not in visited:
                visited.add((nx, ny))
                queue.append((nx, ny))
```

* When deadlock occurs, **reroutes a robot to the nearest empty cell**.
* Uses **BFS** to find the closest free spot.
* Updates grid and robot position accordingly.

---

```python
# Simulation loop
for step in range(STEPS):
    moved = [move_robot(rid, grid) for rid in range(NUM_ROBOTS)]
    
    if not any(moved) or detect_deadlock(grid):
        print("Deadlock detected! Rerouting robots...")
        for rid in range(NUM_ROBOTS):
            reroute_robot(rid, grid)
    
    # Print grid
    for i in range(GRID_SIZE[0]):
        row = ""
        for j in range(GRID_SIZE[1]):
            if grid[i, j] == -1:
                row += ". "
            else:
                row += str(grid[i, j]) + " "
        print(row)
    print("\nStep", step+1, "\n")
```

* Runs **simulation for 50 steps**.
* Tries to move all robots.
* If **deadlock detected**, reroutes robots.
* Prints grid at each step for visualization:

  * `.` → empty cell
  * `0, 1, 2, …` → robot IDs

---

### ✅ **Summary of What Happens**

1. Robots are placed on a 10×10 warehouse grid.
2. Each robot moves toward its target.
3. If robots block each other, **deadlock is detected**.
4. Deadlocked robots are **rerouted to nearest free cell** using BFS.
5. Each step prints the **current warehouse layout**.

---



    

# **100 Interview Questions & Answers Based on the Deadlock Detection Code**

---

## 🟢 Easy Level (Basics & Fundamentals)

1. **Q:** What does the `GRID_SIZE = (10, 10)` represent?
   **A:** The warehouse grid is 10 rows by 10 columns.

2. **Q:** What does `-1` represent in the grid?
   **A:** `-1` means the cell is empty.

3. **Q:** What do numbers like `0, 1, 2, …` in the grid represent?
   **A:** They represent robot IDs.

4. **Q:** How many robots are used in this simulation?
   **A:** 5 robots (`NUM_ROBOTS = 5`).

5. **Q:** How many steps does the simulation run?
   **A:** 50 steps (`STEPS = 50`).

6. **Q:** What Python library is used for grid representation?
   **A:** NumPy (`import numpy as np`).

7. **Q:** Why is `random` imported?
   **A:** To generate random positions for robots and targets.

8. **Q:** What does the `deque` data structure help with?
   **A:** BFS (Breadth-First Search) during rerouting.

9. **Q:** How is the initial robot position decided?
   **A:** Randomly within the grid size.

10. **Q:** What does the `robot_targets` array store?
    **A:** Random target positions for each robot.

11. **Q:** What is the purpose of the `is_valid()` function?
    **A:** To check if a move is inside the grid and not occupied.

12. **Q:** What does `move_robot()` do?
    **A:** Moves a robot one step toward its target if possible.

13. **Q:** Why are moves randomized with `random.shuffle(moves)`?
    **A:** To reduce conflicts between robots.

14. **Q:** What is a deadlock in this simulation?
    **A:** When no robot can move.

15. **Q:** Which function checks for deadlock?
    **A:** `detect_deadlock(grid)`.

16. **Q:** Which function reroutes a robot during deadlock?
    **A:** `reroute_robot(robot_id, grid)`.

17. **Q:** What algorithm does `reroute_robot()` use?
    **A:** Breadth-First Search (BFS).

18. **Q:** What does the simulation print at each step?
    **A:** The grid with robot positions.

19. **Q:** How are robot moves decided?
    **A:** Based on whether the target is above, below, left, or right.

20. **Q:** Why do robots sometimes not move even if the target exists?
    **A:** Because the cell may be occupied by another robot.

21. **Q:** What is stored in `robot_positions`?
    **A:** Current positions of all robots.

22. **Q:** How is the old cell of a robot freed after it moves?
    **A:** By setting it to `-1`.

23. **Q:** How are robots represented in the grid during printing?
    **A:** As numbers (`0–4`) or `.` for empty.

24. **Q:** Why is BFS chosen for rerouting instead of DFS?
    **A:** BFS finds the nearest available cell.

25. **Q:** What happens when deadlock is detected?
    **A:** Robots are rerouted.

26. **Q:** What data type does `robot_positions` use?
    **A:** A NumPy array of robot coordinates.

27. **Q:** Can two robots start at the same cell?
    **A:** No, because the grid prevents overwriting.

28. **Q:** What is `DIRECTIONS` used for?
    **A:** Movement offsets (up, down, left, right).

29. **Q:** How is the grid initialized?
    **A:** `-1 * np.ones(GRID_SIZE, dtype=int)`.

30. **Q:** Why do robots not move diagonally?
    **A:** Because only four directions are allowed.

31. **Q:** What does `not any(moved)` check?
    **A:** If no robot moved in that step.

32. **Q:** Why is deadlock checked in two ways (`not any(moved)` and `detect_deadlock`)?
    **A:** To catch both immediate and potential deadlocks.

33. **Q:** What’s the purpose of `visited` in BFS?
    **A:** To avoid re-checking already visited cells.

34. **Q:** How are targets assigned?
    **A:** Randomly chosen different from robot’s start.

35. **Q:** Does the code guarantee robots reach their targets?
    **A:** No, rerouting may send them elsewhere.

36. **Q:** What does `random.shuffle(moves)` prevent?
    **A:** All robots moving in the same fixed priority.

37. **Q:** Why is `queue = deque([start])` used?
    **A:** To begin BFS from the robot’s current position.

38. **Q:** What happens when BFS finds an empty cell?
    **A:** The robot is moved there immediately.

39. **Q:** What happens if no empty cell is found in BFS?
    **A:** The robot cannot reroute.

40. **Q:** Does the simulation end early if all robots are stuck?
    **A:** No, it always runs for 50 steps.

---

## 🟡 Moderate Level (Logic & Algorithms)

41. **Q:** How does `move_robot()` prioritize moves toward the target?
    **A:** It adds moves in the direction closer to the target first.

42. **Q:** Why does the simulation not always guarantee deadlock resolution?
    **A:** Because rerouted robots might still block each other.

43. **Q:** Could starvation (a robot never reaching its goal) happen?
    **A:** Yes, if robots keep rerouting indefinitely.

44. **Q:** What is the time complexity of `detect_deadlock()`?
    **A:** O(NUM\_ROBOTS × 4), since each robot checks 4 directions.

45. **Q:** What is the time complexity of BFS in `reroute_robot()`?
    **A:** O(N × M), where N and M are grid dimensions.

46. **Q:** Why are targets never guaranteed to be reached?
    **A:** Because rerouting doesn’t consider original targets.

47. **Q:** What could be improved in rerouting to better resolve deadlocks?
    **A:** Reassign robots temporary paths instead of random free cells.

48. **Q:** Why is the simulation deterministic if we remove `random.shuffle`?
    **A:** Because robots would always try moves in the same order.

49. **Q:** What problem does `random.shuffle` solve in terms of deadlock?
    **A:** It reduces the chance of robots making identical moves.

50. **Q:** Can two robots swap places in one step?
    **A:** No, because moves happen sequentially.

51. **Q:** Why is deadlock easier to occur in narrow grid sections?
    **A:** Less free space makes blocking more likely.

52. **Q:** What happens if robots start on their target positions?
    **A:** They won’t move unless blocked.

53. **Q:** Why are robot IDs unique?
    **A:** To track individual robots in the grid.

54. **Q:** What data structure is best suited for BFS?
    **A:** A queue (`deque` in Python).

55. **Q:** Why does rerouting only move robots to the nearest free cell?
    **A:** To quickly break deadlocks.

56. **Q:** Why is `is_valid()` essential in both movement and BFS?
    **A:** To prevent out-of-bounds and collisions.

57. **Q:** How can deadlock detection be extended?
    **A:** By predicting collisions in the next step.

58. **Q:** What does “random target assignment” simulate?
    **A:** Robots having different delivery/pickup points.

59. **Q:** What is the risk of all robots being rerouted at once?
    **A:** They may cause another deadlock.

60. **Q:** How could priorities reduce deadlocks?
    **A:** Assigning some robots priority to move first.

61. **Q:** Why is BFS preferred for rerouting over greedy search?
    **A:** BFS guarantees finding the nearest available space.

62. **Q:** Can robots move outside the grid?
    **A:** No, `is_valid()` prevents that.

63. **Q:** Why is the simulation not “parallel”?
    **A:** Because robots are moved sequentially in a loop.

64. **Q:** Could two robots move into the same empty cell in the same step?
    **A:** No, because the first robot occupies it immediately.

65. **Q:** What would happen if two robots had the same target?
    **A:** They’d block each other near the target.

66. **Q:** Why is there no collision handling in the code?
    **A:** The grid prevents two robots from entering the same cell.

67. **Q:** What is the purpose of `deadlock_detected = False` before each step?
    **A:** To track if movement failed that round.

68. **Q:** What part of the code prevents infinite loops in BFS?
    **A:** The `visited` set.

69. **Q:** What scenario might still cause infinite rerouting loops?
    **A:** Robots chasing each other in cycles.

70. **Q:** What would happen if grid size was smaller than robot count?
    **A:** Robots would overlap, violating uniqueness.

71. **Q:** How does BFS ensure fairness in rerouting?
    **A:** All cells are explored level by level.

72. **Q:** Why doesn’t the code re-check if rerouted robots reached their target?
    **A:** Because rerouting only cares about breaking deadlocks.

73. **Q:** What could happen if BFS didn’t mark visited cells?
    **A:** Infinite looping on the same positions.

74. **Q:** Why is the system reactive instead of preventive?
    **A:** It only reroutes after deadlock, not before.

75. **Q:** Could this system be extended for hundreds of robots?
    **A:** Yes, but performance would degrade without optimization.

76. **Q:** Why are robots allowed to move only one step at a time?
    **A:** To simulate real robot movement.

---

## 🔴 Hard Level (Deep Thinking & Extensions)

81. **Q:** How could this code be extended to detect *potential* deadlocks before they occur?
    **A:** By simulating one step ahead and checking if all moves lead to blocking.

82. **Q:** How would you modify rerouting to always bring a robot closer to its target?
    **A:** Use BFS toward the target instead of the nearest free cell.

83. **Q:** What’s the worst-case time complexity of one simulation step?
    **A:** O(NUM\_ROBOTS × GRID\_SIZE) due to movement + BFS checks.

84. **Q:** Could robots reach deadlock again after rerouting?
    **A:** Yes, especially in crowded grids.

85. **Q:** How can priority scheduling avoid deadlocks?
    **A:** Allow higher-priority robots to move first, forcing others to wait.

86. **Q:** How would you ensure fairness in rerouting?
    **A:** Track rerouted robots and give them cooldown time before rerouting again.

87. **Q:** How could machine learning help optimize rerouting?
    **A:** By predicting better reroute positions from past patterns.

88. **Q:** Why might BFS rerouting be inefficient in large grids?
    **A:** Because BFS explores too many nodes unnecessarily.

89. **Q:** How could A\* search improve robot movement?
    **A:** It finds the optimal path toward the target, not just nearest free cells.

90. **Q:** Why might detecting livelock (constant rerouting) be harder than deadlock?
    **A:** Because robots are moving but making no progress.

91. **Q:** How could path reservation prevent deadlock?
    **A:** Robots reserve future cells before moving.

92. **Q:** How could graph theory be used to model robot conflicts?
    **A:** By representing robots as nodes and conflicts as edges.

93. **Q:** What’s the main scalability issue in this simulation?
    **A:** BFS rerouting is expensive when robot count and grid size grow.

94. **Q:** How would you modify `detect_deadlock()` to handle partial deadlocks (some robots stuck, some moving)?
    **A:** Track individual mobility instead of all-or-nothing.

95. **Q:** Could reinforcement learning replace BFS rerouting?
    **A:** Yes, robots could learn optimal escape policies.

96. **Q:** What real-world constraint is missing in this code?
    **A:** Robots don’t have size or acceleration constraints.

97. **Q:** How could communication between robots reduce deadlocks?
    **A:** Robots could share intended paths to avoid collisions.

98. **Q:** How would you extend this system to multiple floors in a warehouse?
    **A:** Model 3D grid with vertical moves.

99. **Q:** What’s the risk if all robots are rerouted simultaneously?
    **A:** They may just form another deadlock.

100. **Q:** How can centralized scheduling solve deadlock better than local rerouting?
     **A:** A central system assigns conflict-free paths for all robots.

---
